## Datasets Overview
For this capstone project, I will be utilizing several datasets from Yelp and Yummly in identifying correct restaurants through user's search input.

### Yelp Dataset
The [Yelp dataset](https://www.yelp.com/dataset) contains wealth of information on users and businesses spanning 10 metropolitan cities (Montreal, Calgary, Toronto, Pittsburgh, Charlotte, Urbana-Champaign, Phoenix, Las Vegas, Madison, and Cleveland).
- 8,021,122 reviews
- 209,393 businesses
- 1,320,761 tips

Prior to data wrangling, I utilized json_to_csv.py to flatten nested json objects within business' attributes column (link: [json_to_csv_converter](https://github.com/Yelp/dataset-examples/blob/master/json_to_csv_converter.py))

### Yummly Dataset
The [Yummly dataset](https://www.yummly.com/insights/understanding-cuisines) consists of 20 cuisines with various ingredients list per data row. 
- 20 cuisine types
- 39774 ingredients list

In [229]:
import pandas as pd
import numpy as np

# Webscraping for menu data
import requests
from bs4 import BeautifulSoup

# Ignore warning messages
import warnings
warnings.filterwarnings('ignore')

In [230]:
# Utilize helper functions
%run helper_function.ipynb

## Import Files

In [130]:
# Yelp dataset
business_df = pd.read_csv('./dataset/business.csv')
review_df = pd.read_csv('./dataset/review.csv')
tip_df = pd.read_csv('./dataset/tip.csv')

# Yummly dataset
menu_df = pd.read_json('./dataset/cuisine_desc.json')

## 1) Preprocess Yummly Dataset
- Consolidate all ingredient list per cuisine type
- Apply tf-idf vectorizer to consolidated dataset

In [131]:
menu_df.shape

(39774, 3)

In [132]:
menu_df.isnull().any()

id             False
cuisine        False
ingredients    False
dtype: bool

Yummly dataset consists of 20 cuisine types along with its ingredient list.

In [133]:
# Yummly dataset
menu_df.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


Top 5 most accounted cuisines in Yummly dataset are Italian, Mexican, American, Indian, and Chinese

In [134]:
# Yummly dataset
menu_df['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

### 1.1 Consolidate all ingredients list per cuisine type
- Consolidating all text values per cuisine in preparation for tf-idf vectorizer

In [135]:
# Convert row ingredients as string
menu_df['item_list'] = menu_df['ingredients'].apply(lambda row: ','.join(map(str, row)))

# Consolidate all ingredients using groupby method
menu_df = menu_df.groupby('cuisine', as_index=False).agg({'item_list': ' '.join})

In [136]:
menu_df

,cuisine,item_list
0,brazilian,"ice cubes,club soda,white rum,lime,turbinado e..."
1,british,"greek yogurt,lemon curd,confectioners sugar,ra..."
2,cajun_creole,"herbs,lemon juice,fresh tomatoes,paprika,mango..."
3,chinese,"low sodium soy sauce,fresh ginger,dry mustard,..."
4,filipino,"eggs,pepper,salt,mayonaise,cooking oil,green c..."
5,french,"sugar,salt,fennel bulb,water,lemon olive oil,g..."
6,greek,"romaine lettuce,black olives,grape tomatoes,ga..."
7,indian,"water,vegetable oil,wheat,salt black pepper,sh..."
8,irish,"cooking spray,salt,black pepper,yukon gold pot..."
9,italian,"sugar,pistachio nuts,white almond bark,flour,v..."


### 1.2 Apply TF-IDF vectorizer to consolidated Yummly dataset
TF-IDF vectorizer is used to identify words or ingredients that have more at certain cuisine type while not appearing on other cuisines. It'll be useful when each cuisines' have its own relevant keywords so that during machine learning phase (word2vec, doc2vec, etc.); we can utilize this dataset to accurately respond to user's search request.
- Create vocabulary for tfidf
- Invoke helper function to create two menu tfidf's dataframes
    - Create one with using menu list provided by Yummly's dataset
    - Create second tfidf matrix dataframe using n-gram

#### 1.2.1 Create vocabulary for tfidf

In [137]:
# Create menu lists by converting concatenated string to list
menu_lists = []
menu_df['item_list'].apply(lambda text: menu_lists.append(text.split(',')))

# Flatten nested lists
menu_list = [item.strip() for menu_list in menu_lists for item in menu_list]

# Get unique values
menu_list = list(set(menu_list))

# Sample
menu_list[:10]

['',
 'corn starch ketchup',
 'mint leaves',
 '2% reduced-fat milk butter',
 'olive oil cooking spray black pepper',
 'cooked brisket pepper',
 'large eggs white vinegar',
 'heavy cream baby arugula',
 'dried oregano marsala wine',
 'egg whites']

In [138]:
# Remove first index which is an empty string
menu_list.pop(0)

''

#### 1.2.2 Invoke helper function to create menu tfidf's dataframes

#### Menu TF-IDF dataframe based on Yummly's given vocabulary

In [139]:
# Invoking function to return tfidf matrix dataframe
menu_tfidf_df = create_tfidf_matrix_df(menu_df, menu_list)

In [140]:
# Viewing result
menu_tfidf_df

,corn starch ketchup,mint leaves,2% reduced-fat milk butter,olive oil cooking spray black pepper,cooked brisket pepper,large eggs white vinegar,heavy cream baby arugula,dried oregano marsala wine,egg whites,white grapefruit,...,bay leaf minced garlic,chipotles in adobo lime juice,almond extract ground nutmeg,dried cranberries chopped tomatoes,mushrooms sugar,chopped cilantro diced onions,fresh lemon juice chicken stock,garlic cloves granulated sugar,basmati rice diced onions,vanilla extract fresh spinach
cuisine,,,,,,,,,,,,,,,,,,,,,
brazilian,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
british,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cajun_creole,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chinese,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
filipino,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
french,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
greek,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
indian,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
irish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Checking the tf-idf result provided by Yummly's vocabulary

In [141]:
menu_tfidf_df.loc['indian'].sort_values(ascending=False)[:10]

salt         0.378676
oil          0.321795
garlic       0.259369
cumin        0.247644
masala       0.246614
ginger       0.233702
pepper       0.223878
onions       0.205024
coriander    0.183000
cilantro     0.165413
Name: indian, dtype: float64

In [142]:
menu_tfidf_df.loc['korean'].sort_values(ascending=False)[:10]

sesame    0.432001
oil       0.374249
sauce     0.308584
garlic    0.293430
onions    0.258071
soy       0.249806
pepper    0.246591
sugar     0.224090
rice      0.190109
salt      0.174497
Name: korean, dtype: float64

In [143]:
menu_tfidf_df.loc['mexican'].sort_values(ascending=False)[:10].keys().tolist()

['pepper',
 'salt',
 'garlic',
 'cheese',
 'oil',
 'cilantro',
 'chicken',
 'lime',
 'onions',
 'tortillas']

#### Identifying top 100 ingredients in all 20 cuisines

In [144]:
data = {}
for cuisine in menu_tfidf_df.index:
    # Top 100 ingredients that have high tf-idf score
    values = menu_tfidf_df.loc[cuisine].sort_values(ascending=False)[:100].keys().tolist()
    data[cuisine] = values

menu_vocab_tfidf_df = pd.DataFrame(data,columns=menu_tfidf_df.index.tolist())

In [145]:
menu_vocab_tfidf_df

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
0,pepper,salt,pepper,sauce,sauce,salt,pepper,salt,salt,cheese,pepper,sauce,sesame,pepper,pepper,salt,salt,pepper,sauce,sauce
1,oil,flour,garlic,oil,garlic,pepper,oil,oil,flour,pepper,salt,oil,oil,salt,oil,sugar,pepper,oil,lime,sugar
2,salt,sugar,salt,soy,pepper,butter,salt,garlic,butter,oil,garlic,mirin,sauce,garlic,salt,flour,sugar,salt,oil,garlic
3,cachaca,butter,onions,garlic,salt,sugar,lemon,cumin,sugar,salt,thyme,soy,garlic,cheese,garlic,butter,butter,garlic,garlic,oil
4,milk,eggs,oil,pepper,oil,oil,garlic,masala,pepper,garlic,oil,sugar,onions,oil,cumin,pepper,flour,red,fish,fish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,kale,crust,nutmeg,loin,paprika,asparagus,pastry,chiles,rye,wheat,sage,tuna,chuck,honey,oregano,radishes,margarine,apples,linguine,papaya
96,soda,chives,cilantro,almonds,parsley,sage,salad,spices,oatmeal,seeds,cabbage,steak,perilla,beer,cauliflower,ketchup,sausages,starch,mussels,avocado
97,almonds,canola,crabmeat,cayenne,breast,veal,flatbread,peanuts,chips,porcini,ice,crab,gingerroot,wheat,stew,cilantro,pimentos,hazelnuts,chestnuts,coffee
98,scallions,leeks,rolls,spinach,macaroni,shrimp,marjoram,beef,syrup,ravioli,browning,beans,roast,lard,spinach,shallots,bananas,salsa,cardamom,pineapple


In [146]:
menu_vocab_tfidf_df.to_csv('./dataset/menu_vocab_tfidf.csv')

Based on the results given by Menu TF-IDF's dataframe based on Yummly's vocabulary set - the result seemed general. For instance 'garlic', 'salt', and 'pepper' are visible on all three cuisines. It did not provide any unique ingredient per cuisine as I had hoped. Therefore, I'll be creating a second Menu tf-idf dataframe based on n-gram.

#### Menu TF-IDF dataframe based on N-gram

In [147]:
# Invoking function to return tfidf matrix dataframe
menu_tfidf_df = create_tfidf_matrix_df(menu_df)

In [148]:
# Viewing result
menu_tfidf_df

,10,10 oz,10 oz frozen,14,14 oz,14 oz diced,14 oz sweetened,25,25 less,25 less sodium,...,zucchini vegetable,zucchini vegetable broth,zucchini vegetable oil,zucchini vegetable stock,zucchini whole,zucchini yellow,zucchini yellow bell,zucchini yellow onion,zucchini yellow squash,épices
cuisine,,,,,,,,,,,,,,,,,,,,,
brazilian,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
british,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cajun_creole,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001773,0.001773,0.001773,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
chinese,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
filipino,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.004968,0.000000,0.005695,0.000000,0.000000
french,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003798,0.002503,0.002316,0.000000,0.000000,0.002020,0.003028,0.000000,0.000000,0.006056
greek,0.002727,0.002727,0.002727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
indian,0.000000,0.000000,0.000000,0.000864,0.000864,0.000000,0.000958,0.000000,0.000000,0.000000,...,0.001201,0.000792,0.000000,0.000958,0.000000,0.001916,0.000000,0.001465,0.000000,0.000000
irish,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.004734,0.000000,0.005773,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


TF-IDF model with n_gram range from 1 to 3 - produced 365373 columns which is a lot more than previous vocab based TF-IDF dataframe of 33727 columns

#### Checking the TF-IDF result based on ngrams

In [149]:
menu_tfidf_df.loc['indian'].sort_values(ascending=False)[:10]

garam masala         0.462462
garam                0.462462
curry leaves         0.201379
dal                  0.157047
ghee                 0.146125
coriander powder     0.120609
garam masala salt    0.108011
fenugreek            0.107102
cardamom pods        0.105446
masala salt          0.102902
Name: indian, dtype: float64

In [150]:
menu_tfidf_df.loc['korean'].sort_values(ascending=False)[:10]

gochujang               0.339206
gochujang base          0.339206
kimchi                  0.279347
toasted sesame seeds    0.206008
sauce sesame            0.169717
soy sauce sesame        0.160074
toasted sesame oil      0.148502
sauce sesame oil        0.129216
mirin                   0.128364
korean                  0.117417
Name: korean, dtype: float64

In [151]:
menu_tfidf_df.loc['mexican'].sort_values(ascending=False)[:10]

taco               0.288057
taco seasoning     0.259648
enchilada sauce    0.232524
enchilada          0.232524
jack cheese        0.214434
mexican            0.207057
refried beans      0.165673
refried            0.153691
tortilla chips     0.153636
tortilla           0.145080
Name: mexican, dtype: float64

#### Identifying top 100 ingredients in all 20 cuisines

In [152]:
data = {}
for cuisine in menu_tfidf_df.index:
    # Top 100 ingredients that have high tf-idf score
    values = menu_tfidf_df.loc[cuisine].sort_values(ascending=False)[:100].keys().tolist()
    data[cuisine] = values

menu_tfidf_df = pd.DataFrame(data,columns=menu_tfidf_df.index.tolist())

In [153]:
menu_tfidf_df

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
0,dried black beans,stilton,cajun,chinese,calamansi,gruyere cheese,feta cheese crumbles,garam masala,irish,lasagna,jerk,mirin,gochujang,taco,couscous,sauerkraut,grits,chorizo,thai,beansprouts
1,dried black,stilton cheese,cajun seasoning,oyster sauce,soy sauce bay,gruyere,cheese crumbles,garam,irish whiskey,lasagna noodles,jamaican,sake,gochujang base,taco seasoning,preserved lemon,buckwheat flour,whiskey,saffron threads,curry paste,thai
2,chocolate sprinkles,suet,andouille,shaoxing,oyster sauce,fresh tarragon,kalamata,curry leaves,irish cream liqueur,parmigiano,scotch bonnet,miso,kimchi,enchilada sauce,hanout,cottage cheese,bourbon,chorizo sausage,red curry,rice noodles
3,granola,currants,creole,hoisin,thai chile,swiss cheese,kalamata olives,dal,irish cream,parmigiano reggiano,bonnet chile,dashi,toasted sesame seeds,enchilada,ras,cottage,bourbon whiskey,spanish chorizo,red curry paste,vietnamese
4,ice lime,golden syrup,andouille sausage,hoisin sauce,thai,grated gruyère,pita,ghee,whiskey,reggiano cheese,scotch bonnet chile,nori,sauce sesame,jack cheese,ras el,buckwheat,collard greens,manchego cheese,lime leaves,thai basil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eggs parmesan,soda vanilla,onion powder dried,sesame oil rice,banana leaves,frisee,oregano black pepper,seeds cumin,fashioned,salt fresh basil,baking powder cornmeal,chili oil,sugar rice,ground cumin avocado,green olives olive,butter golden,chicken soup,salt manchego,sauce thai basil,sauce fresh ginger
96,passion fruit juice,sugar dark,powder dried oregano,sauce soy sauce,garlic coconut milk,chocolate large,yogurt garlic,salt ghee,wheat flour buttermilk,porcini,scallions coconut milk,soy sauce fresh,seeds green,flour tortillas salt,cinnamon lemon,potato starch,meal large,red peppers extra,paste lime,sauce fish
97,milk tapioca flour,potatoes all purpose,creole seasoning celery,sauce pork,sauce rice noodles,duck fat,pepper greek,jeera,carrots beef brisket,italian bread,scallions coconut,rice rice vinegar,reduced sodium soy,old el paso,lamb stock,potatoes all,cracker crumbs,tomatoes dry white,light soy,cilantro rice
98,ice granulated,sugar whipping cream,cajun seasoning all,rice vinegar corn,soy sauce vinegar,pernod,kalamata feta,gram flour,bacon pork sausages,cheese italian seasoning,thyme hot,green tea powder,soy sauce vegetable,el paso,ground coriander chopped,tomatoes potatoes,marshmallows,hot smoked paprika,light soy sauce,sesame oil rice


In [154]:
menu_tfidf_df.to_csv('./dataset/menu_tfidf.csv')

Menu TF-IDF's dataframe based on ngrams approach provided better result in identifying list of less general ingredients per cuisine types.

### Summary of Yummly dataset
- Initial Yummly's dataset contains 20 cuisines with 39,774 rows with food ingredients.
- Consolidated Yummly's dataset based on cuisine type which reduced row count to 20 (cuisine count total)
- Applied TF-IDF Transformer onto consolidated Yummly's dataset in which n_grams (unigrams to trigrams) provided better distinction between cuisines compared to given Yummly's menu titles.

---

## 2) Extracting menu data through webscraping
I decided to use BeautfiulSoup to webscrape allmenus.com as it contains restaurant name, cuisine type, menu item, and its menu description. All the data needed to help supplement our current datasets in recommending restaurants to the users based on their search input.

1. Remove businesses from Canada and lowercase all city and state names.
2. Collect restaurants menu from allmenus.com using business dataframe's states.
3. Based on given restaurants name from allmenus, filter both allmenus and yelp's business dataset.

### 2.1 Remove businesses from Canada and lowercase all city names
Yelp dataset contains businesses mostly from USA and some from Canada, I am filtering out cities from Canada as I'm focusing on restaurants from USA.

In [155]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [156]:
# Get businesses from US states
business_df = business_df[business_df['state'].isin(states)]

In [157]:
business_df['state'].unique()

array(['NC', 'AZ', 'NV', 'IL', 'PA', 'WI', 'SC', 'OH', 'CA', 'TX', 'NY',
       'CO', 'GA', 'AL', 'UT', 'VT', 'WA', 'NE', 'MI', 'FL', 'AR', 'HI',
       'OR', 'AK', 'VA', 'CT', 'MO'], dtype=object)

In [158]:
# Lowercasing city names and states to be case-insensitive
business_df["city"] = business_df["city"].str.lower()
business_df['state'] = business_df['state'].str.lower()

Gathered businesses from 27 US states and its cities from Yelp's dataset.

### 2.2 Collect restaurants menu from allmenus.com using business dataframe's states
Collect top 5 cities from filtered states data and begin webscraping for restaurants' menu.

In [159]:
# Check business counts in five major cities in Nevada
data = []
states = business_df['state'].unique()

for state in states:
    
    # Get cities per state
    cities = business_df[business_df['state'] == state].groupby(by=['city'])
    
    # Get top 5 cities per state
    cities = cities['city'].count().sort_values(ascending=False)[:5].keys().tolist()
    
    # Create object per each state
    obj = {'state': state, 'cities': cities}
    
    # Append each state's cities data
    data.append(obj)

In [160]:
# Viewing sample data
data[:3]

[{'state': 'nc',
  'cities': ['charlotte', 'concord', 'matthews', 'huntersville', 'gastonia']},
 {'state': 'az',
  'cities': ['phoenix', 'scottsdale', 'mesa', 'tempe', 'chandler']},
 {'state': 'nv',
  'cities': ['las vegas',
   'henderson',
   'north las vegas',
   'boulder city',
   'n las vegas']}]

#### Extract restaurant data from allmenus.com
- Get restaurants' sub url links and its restaurant titles from allmenus.com
- Build menu dataframe using restaurants' links and titles

#### Get restaurants' sub url links and its titles

In [161]:
import pickle # Saving lists or data from allmenus.com

In [166]:
rest_links, rest_titles = collect_menu_links(data) # collects restaurant links from allmenus.com

In [169]:
# Saving files for later use
with open('./dataset/rest_links.pkl', 'wb') as f:
    pickle.dump(rest_links, f)
    
with open('./dataset/rest_titles.pkl', 'wb') as f:
    pickle.dump(rest_titles, f)

#### Remove states that did not appear in allmenus.com

In [230]:
# Divide restaurant links by state
links = [] # Append list per state
unavailable_states = [] # collect unavailable states

for state in states:
    temp_links = [link for link in rest_links if link.split('/')[1] == state]
    
    # Check if there are states not available in allmenus.com
    if len(temp_links) == 0:
        unavailable_states.append(state)
    else:
        links.append(temp_links)    

In [717]:
unavailable_states

['al', 'vt', 'ar', 'hi', 'ak']

In [718]:
# Remove five states that are not used in allmenus.com
updated_states = [state for state in states if state not in unavailable_states]

#### Organize restaurant links and titles by state using dictionary

In [233]:
links = [] # dividing restaurant links per state
data = {}

for state in updated_states:
    rest_list = [link for link in rest_links if link.split('/')[1] == state] # get restaurant links per state
    titles = rest_titles[:len(rest_list)] # get restaurant titles per state
    
    data[state] = [rest_list, titles] # key value pair - state<key> restaurant links and titles <value>
    rest_titles = rest_titles[len(rest_list):] # update list by removing recently added restaurant titles

#### Build menu dataframe using restaurant links and titles
For some reason, could not make requests all the restaurant links/titles data at once. Therefore, dividing requests by state.

In [247]:
nc_menu_df = build_menu_df(data['nc'][0], data['nc'][1]) # Build menus dataframe

In [243]:
menu_df = build_menu_df(data['nv'][0], data['nv'][1]) # Build menus dataframe

In [ ]:
nv_menu_df = menu_df

In [251]:
az_menu_df = build_menu_df(data['az'][0], data['az'][1]) # Build menus dataframe

In [254]:
il_menu_df = build_menu_df(data['il'][0], data['il'][1]) # Build menus dataframe

In [255]:
pa_menu_df = build_menu_df(data['pa'][0], data['pa'][1]) # Build menus dataframe

In [256]:
wi_menu_df = build_menu_df(data['wi'][0], data['wi'][1]) # Build menus dataframe

In [257]:
sc_menu_df = build_menu_df(data['sc'][0], data['sc'][1]) # Build menus dataframe

In [258]:
oh_menu_df = build_menu_df(data['oh'][0], data['oh'][1]) # Build menus dataframe

In [259]:
ca_menu_df = build_menu_df(data['ca'][0], data['ca'][1]) # Build menus dataframe

In [264]:
ny_menu_df = build_menu_df(data['ny'][0], data['ny'][1]) # Build menus dataframe

In [265]:
co_menu_df = build_menu_df(data['co'][0], data['co'][1]) # Build menus dataframe

In [266]:
ga_menu_df = build_menu_df(data['ga'][0], data['ga'][1]) # Build menus dataframe

In [267]:
ut_menu_df = build_menu_df(data['ut'][0], data['ut'][1]) # Build menus dataframe

In [268]:
wa_menu_df = build_menu_df(data['wa'][0], data['wa'][1]) # Build menus dataframe

In [269]:
ne_menu_df = build_menu_df(data['ne'][0], data['ne'][1]) # Build menus dataframe

In [270]:
mi_menu_df = build_menu_df(data['mi'][0], data['mi'][1]) # Build menus dataframe

In [271]:
fl_menu_df = build_menu_df(data['fl'][0], data['fl'][1]) # Build menus dataframe

In [272]:
or_menu_df = build_menu_df(data['or'][0], data['or'][1]) # Build menus dataframe

In [273]:
va_menu_df = build_menu_df(data['va'][0], data['va'][1]) # Build menus dataframe

In [274]:
ct_menu_df = build_menu_df(data['ct'][0], data['ct'][1]) # Build menus dataframe

In [275]:
mo_menu_df = build_menu_df(data['mo'][0], data['mo'][1]) # Build menus dataframe

After successfully webscraping all restaurants from the allmenus website, concat all menu dataframes into one and save consolidated menu dataframe.

In [300]:
# Concat
frames = [nc_menu_df, nv_menu_df, az_menu_df, il_menu_df, pa_menu_df, wi_menu_df, sc_menu_df, oh_menu_df, 
          ca_menu_df, ny_menu_df, co_menu_df, ga_menu_df, ut_menu_df, wa_menu_df, ne_menu_df, mi_menu_df, 
          fl_menu_df, or_menu_df, va_menu_df, ct_menu_df, mo_menu_df]

allmenus_df = pd.concat(frames)
allmenus_df.head()

,id,name,menu_titles,menu_desc,categories
0,32810,El Cancun,Hamburger with fries,,Mexican
1,32810,El Cancun,Child's Plate #1,One taco&comma rice & beans,Mexican
2,32810,El Cancun,Child's Plate #2,Mini burrito&comma rice & beans,Mexican
3,32810,El Cancun,Chicken Plate,Grilled strips of chicken & fries,Mexican
4,32810,El Cancun,French Fries,,Mexican


In [164]:
allmenus_df.shape

(1354448, 5)

Webscraping all restaurants' list from 22 states have generated 5 columns with 1,354,448 datapoints. 

In [165]:
# Lowercase restaurants' names from both yelp and allmenus dataset
allmenus_df['name'] = allmenus_df['name'].str.lower()
business_df['name'] = business_df['name'].str.lower()

In [166]:
# Get unique restaurant names (remove duplicates)
restaurants_list = list(set(allmenus_df['name'].tolist()))

# Have 8875 unique restaurants
len(restaurants_list)

8875

In [167]:
business_df.shape

(153843, 60)

#### Save accumulated allmenus dataset

In [313]:
allmenus_df.to_csv('./dataset/menu.csv')

## 2.3 Filter both Yelp's business dataset and Allmenus' menu dataset
 - Only populate restaurants that exist in both Yelp's business dataset and Allmenus' menu dataset.
 - Combine all text data into one row per restaurants for both Yelp's business and Allmenus' menu dataset.

### 2.3.1 Only populate restaurants that exist in both Yelp's business dataset and Allmenus' menu dataset

In [168]:
# Filter businesses dataframe based on given allmenus' restaurants list
business_df = business_df[business_df['name'].isin(restaurants_list)]

# Filter restaurants' name from allmenus dataset compared to Yelp's business dataset
filtered_allmenus_df = allmenus_df[allmenus_df['name'].isin(business_df['name'])]

In [169]:
business_df.shape

(3404, 60)

In [170]:
filtered_allmenus_df.shape

(313954, 5)

In [171]:
len(set(filtered_allmenus_df.name.tolist()))

1199

In [172]:
len(set(business_df['name'].tolist()))

1199

Restaurants that appeared on both allmenus and Yelp's business dataset resulted in the following:
    - Removed 150,439 restaurant data from Yelp's business dataset (97% reduction).
    - Removed 1,040,494 restaurants from allmenus data (76.8% reduction).
    - Resulted in having 1199 unique restaurants.

### 2.3.2 Combine all text data into one row per restaurants for both Yelp's business and Allmenus' menu dataset
- Filtered Allmenus' menu dataset will combine menu_titles and menu_desc into one row per restaurant.
- Clean Yelp's business dataset before combining all rows per each restaurant.
- Handle NaNs and missing values if any.
- Identify duplicated rows and combine further.
    - Remove duplicate categories within each row (restaurant).
- Create new columns based on categories values - set default value as 0.

#### Consolidating Allmenus' menu dataframe based on name and categories

In [173]:
filtered_allmenus_df

,id,name,menu_titles,menu_desc,categories
521,37789,cabo fish taco,Queso Dip,Piping hot melted cheese blended with chipotle...,"Mexican, Seafood"
522,37789,cabo fish taco,Guacamole,"A house blend of avocado, lime juice, cilantro...","Mexican, Seafood"
523,37789,cabo fish taco,Crispy Calamari,"Beer battered calamari rings, flash-fried and ...","Mexican, Seafood"
524,37789,cabo fish taco,Baja Shrimp and Crab Dip,Creamy dip made with lump crabmeat and seasone...,"Mexican, Seafood"
525,37789,cabo fish taco,Salsa Trio,"Roasted Red Tomato Salsa, fresh Pineapple Mang...","Mexican, Seafood"
...,...,...,...,...,...
6546,52138,fuji,Chicken,NaN,Japanese
6547,52138,fuji,Filet Mignon,NaN,Japanese
6548,52138,fuji,Shrimp & Scallops,NaN,Japanese
6549,52138,fuji,Steak & Chicken,NaN,Japanese


#### Handling Missing Values
Since we will be combining all rows based on restaurant name - I'll be adding empty string on all cells with NaNs.

In [174]:
filtered_allmenus_df[filtered_allmenus_df.isna().any(axis=1)]

,id,name,menu_titles,menu_desc,categories
564,37789,cabo fish taco,Chicken Fajita,NaN,"Mexican, Seafood"
565,37789,cabo fish taco,Shrimp Fajita,NaN,"Mexican, Seafood"
566,37789,cabo fish taco,Steak Fajita,NaN,"Mexican, Seafood"
567,37789,cabo fish taco,Veggie Fajita,NaN,"Mexican, Seafood"
568,37789,cabo fish taco,Salmon Baja Bowl,NaN,"Mexican, Seafood"
...,...,...,...,...,...
6546,52138,fuji,Chicken,NaN,Japanese
6547,52138,fuji,Filet Mignon,NaN,Japanese
6548,52138,fuji,Shrimp & Scallops,NaN,Japanese
6549,52138,fuji,Steak & Chicken,NaN,Japanese


In [175]:
filtered_allmenus_df.isna().sum()

id                  0
name                0
menu_titles         1
menu_desc      119635
categories          0
dtype: int64

In [176]:
filtered_allmenus_df = filtered_allmenus_df.fillna('')
filtered_allmenus_df.isna().sum()

id             0
name           0
menu_titles    0
menu_desc      0
categories     0
dtype: int64

### Text preprocessing - tokenizing, lemmatizing, lowercasing, and removing stop words.

In [177]:
filtered_allmenus_df['clean_menu_desc'] = filtered_allmenus_df['menu_desc'].apply(lambda text: tokenize(text))

In [178]:
filtered_allmenus_df['clean_menu_titles'] = filtered_allmenus_df['menu_titles'].apply(lambda text: tokenize(text))

#### Aggregating all text values grouped by restaurant name

In [179]:
# Consolidate all menu description and menu titls using groupby method
filtered_allmenus_df = filtered_allmenus_df.groupby(['name', 'categories'], as_index=False).agg({'menu_desc': ' '.join, 'menu_titles': ' '.join, 'clean_menu_desc': ' '.join, 'clean_menu_titles': ' '.join})
filtered_allmenus_df.head()

,name,categories,menu_desc,menu_titles,clean_menu_desc,clean_menu_titles
0,#1 pho,Vietnamese,Finely ground pork grilled on wooden skewer wi...,Grilled Meat Balls Vietnames Crispy Crepe Cris...,finely ground pork grill wooden skewer lettuce...,grilled meat ball vietname crispy crepe crispy...
1,24th street pizza & gyros,"Italian, Pizza, Sandwiches, American, Wings","Tomatoes, onions and tzatziki sauce. Lettuce, ...",Gyros Pita Chicken Gyros Pita Calzone Fried Zu...,tomato onion tzatziki sauce lettuce tomato oni...,gyros pita chicken gyros pita calzone fried zu...
2,3 tomatoes & a mozzarella,"Italian, Pizza",Small side salad of greens&comma dressed with ...,Field Greens Caesar Mediterranean Spinach & Ar...,small salad dress house balsamic vinaigrette r...,field greens caesar mediterranean spinach arug...
3,4b cafe,"Mexican, American",stuffed with cream cheese. our freshly...,Chips And Salsa Chips And Nacho Cheese Sauce C...,stuff cream cheese freshly fry corn to...,chip and salsa chip and nacho cheese sauce chi...
4,5 r cha thai go,Thai,Finely sliced veggies and glass noodles wr...,Thai Iced Tea Thai Iced Coffee Iced Tea Soft D...,finely sliced veggie glass noodle wrap lig...,thai iced tea thai iced coffee iced tea soft d...


There are no missing values.

In [180]:
# Determine if ANY value in a series is missing
filtered_allmenus_df.isnull().values.any()

False

#### Identify duplicated rows and combine further
Analyze duplicated restaurants and combine further based on restaurant name.

In [181]:
filtered_allmenus_df[filtered_allmenus_df['name'].duplicated()]

,name,categories,menu_desc,menu_titles,clean_menu_desc,clean_menu_titles
39,asian kitchen,"Chinese, Asian",1 piece. 2 pieces. 8 pieces. Fried or steame...,8. Egg Roll 9. Chicken Egg Roll 10. Pork Egg R...,piece piece piece fried steam choice piece...,egg roll chicken egg roll pork egg roll spring...
44,athens gyros,"Greek, Salads, Healthy","Mediterranean dip garbanzo beans, s...",Lipton Green Tea Lipton Iced Tea Bottled Water...,mediterranean dip garbanzo beans se...,lipton green tea lipton iced tea bottled water...
52,bacaro,"Italian, Pizza, Alcohol, Tapas",Served with garlic olive oil. Served...,3 Cheese and Salumi 5 Cheese and Salumi Apple ...,serve garlic olive oil serve marcona...,cheese salumi cheese salumi apple medjool date...
85,benihana,"Japanese, Sushi, Steak",1040-1120 calories. 0 calories. 0-170 calorie...,Benihana Lemonade 1/2 Gallon Freshly Brewed Ic...,calorie calorie calorie oz calorie oz calori...,benihana lemonade gallon freshly brewed ice te...
98,blossom,"Noodles, Vietnamese","Battered and fried sweet potatoes, tofu and mu...",Vegetarian Sweet Potatoes Shrimp Sweet Potatoe...,battered fry sweet potato tofu mushroom serve ...,vegetarian sweet potatoes shrimp sweet potatoe...
...,...,...,...,...,...,...
1301,wahoo's fish taco,"Southwestern, Mexican","Chopped chicken breast, cream cheese, spinach ...",Baja Rolls Cheese Quesadilla French Fries Chip...,chop chicken breast cream cheese spinach salsa...,baja rolls cheese quesadilla french fry chips ...
1305,wasabi sushi,"Japanese, Sushi",334 mL Okinawa Beer. Must be 21 to purchase Be...,Orion Must be 21 to purchase Sapporo Must be 2...,ml okinawa beer must purchase beer sapporo jap...,orion must purchase sapporo must purchase kiri...
1318,yama,"Japanese, Sushi",Baked Japanese eggplant with sweet miso paste...,Soft Drink Nasu Shigiyaki Edamame Wakame Salad...,baked japanese eggplant sweet miso paste japa...,soft drink nasu shigiyaki edamame wakame salad...
1320,yamato,"Japanese, Sushi, Asian Fusion","Golden yellow hued long gin tea, fres...",Can Soda Juice Fiji Still Water Fiji Water Sna...,golden yellow hue long gin tea fresh ...,can soda juice fiji still water fiji water sna...


In [182]:
# Filtered further by name while aggregating menu description/titles and categories
filtered_allmenus_df = filtered_allmenus_df.groupby(
    ['name'], as_index=False).agg(
    {'menu_desc': ' '.join, 
     'menu_titles': ' '.join,
     'clean_menu_desc': ' '.join, 
     'clean_menu_titles': ' '.join,
     'categories': ' '.join})

filtered_allmenus_df.head()

,name,menu_desc,menu_titles,clean_menu_desc,clean_menu_titles,categories
0,#1 pho,Finely ground pork grilled on wooden skewer wi...,Grilled Meat Balls Vietnames Crispy Crepe Cris...,finely ground pork grill wooden skewer lettuce...,grilled meat ball vietname crispy crepe crispy...,Vietnamese
1,24th street pizza & gyros,"Tomatoes, onions and tzatziki sauce. Lettuce, ...",Gyros Pita Chicken Gyros Pita Calzone Fried Zu...,tomato onion tzatziki sauce lettuce tomato oni...,gyros pita chicken gyros pita calzone fried zu...,"Italian, Pizza, Sandwiches, American, Wings"
2,3 tomatoes & a mozzarella,Small side salad of greens&comma dressed with ...,Field Greens Caesar Mediterranean Spinach & Ar...,small salad dress house balsamic vinaigrette r...,field greens caesar mediterranean spinach arug...,"Italian, Pizza"
3,4b cafe,stuffed with cream cheese. our freshly...,Chips And Salsa Chips And Nacho Cheese Sauce C...,stuff cream cheese freshly fry corn to...,chip and salsa chip and nacho cheese sauce chi...,"Mexican, American"
4,5 r cha thai go,Finely sliced veggies and glass noodles wr...,Thai Iced Tea Thai Iced Coffee Iced Tea Soft D...,finely sliced veggie glass noodle wrap lig...,thai iced tea thai iced coffee iced tea soft d...,Thai


In [183]:
filtered_allmenus_df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198],
           dtype='int64', length=1199)

#### Remove duplicate categories (ex: 'Chinese Chinese, Japanese')
Categories column has duplicate category; example shown below:

In [184]:
# Add Allmens' categories onto business dataset matching by restaurant name
filtered_allmenus_df[filtered_allmenus_df['name'] == 'china gourmet']

,name,menu_desc,menu_titles,clean_menu_desc,clean_menu_titles,categories
197,china gourmet,6 pieces. 6 pieces. 6 pieces. 4 p...,20 oz. Coke Pepsi 20 oz. Dr Pepper 20 oz. Diet...,piece piece piece piece piece pi...,oz coke pepsi oz dr pepper oz diet dr pepper o...,"Chinese Chinese, Japanese"


In [185]:
def get_unique_categories(categories):
    '''
    categories: str
        category list per restaurant
    Returns
    -------
    str : str
        Returns categories without duplicates
    '''
    value = categories.replace(',', ' ').split(' ') # Remove comma and split by space
    set_val = set(value) # Remove duplicates
    str_val = " ".join(map(str,set_val)) # Convert Set to string
    return str_val.strip() # Remove white space

In [186]:
# Apply that function to every row of the column
filtered_allmenus_df['categories'] = filtered_allmenus_df['categories'].apply(lambda x: get_unique_categories(x))

In [187]:
# Lowercase categories
filtered_allmenus_df['categories'] = filtered_allmenus_df['categories'].str.lower()
filtered_allmenus_df['categories']

0                                    vietnamese
1       sandwiches wings pizza american italian
2                                 pizza italian
3                              mexican american
4                                          thai
                         ...                   
1194              mexican american southwestern
1195              mexican american southwestern
1196                                     indian
1197                                    chinese
1198          sandwiches pizza american italian
Name: categories, Length: 1199, dtype: object

Removed 142 duplicates by grouping futher with restaurant name.

#### Create new columns based on categories values - set default value as 0

In [188]:
from collections import Counter 

# Creating new columns based on categorical values (ex: american, italian, japanese, etc.)
categories = []

for cat in filtered_allmenus_df['categories']:
    for category in cat.split(' '):
        
        category = category.strip()
        
        if category != '' and category != '&':
            categories.append(category)
            
features = Counter(categories).most_common()
features[:10] # Show top 10 categories

[('chinese', 308),
 ('italian', 269),
 ('mexican', 239),
 ('asian', 155),
 ('american', 149),
 ('pizza', 142),
 ('japanese', 130),
 ('thai', 94),
 ('sandwiches', 89),
 ('sushi', 89)]

In [228]:
# Saving files for later use
with open('./dataset/features.pkl', 'wb') as f:
    pickle.dump(features, f)

In [189]:
# Get unique categories
categories = list(set(categories))

# Concating dataframe with business_df
filtered_allmenus_df = pd.concat(
    [
        filtered_allmenus_df,
        pd.DataFrame(
            index = filtered_allmenus_df.index,
            columns = categories
        )
    ], axis = 1
)

filtered_allmenus_df.head()

,name,menu_desc,menu_titles,clean_menu_desc,clean_menu_titles,categories,vegan,asian,ribs,fine,...,fresh,puerto,gyros,cantonese,european,cream,cocktails,pub,indian,steak
0,#1 pho,Finely ground pork grilled on wooden skewer wi...,Grilled Meat Balls Vietnames Crispy Crepe Cris...,finely ground pork grill wooden skewer lettuce...,grilled meat ball vietname crispy crepe crispy...,vietnamese,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24th street pizza & gyros,"Tomatoes, onions and tzatziki sauce. Lettuce, ...",Gyros Pita Chicken Gyros Pita Calzone Fried Zu...,tomato onion tzatziki sauce lettuce tomato oni...,gyros pita chicken gyros pita calzone fried zu...,sandwiches wings pizza american italian,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3 tomatoes & a mozzarella,Small side salad of greens&comma dressed with ...,Field Greens Caesar Mediterranean Spinach & Ar...,small salad dress house balsamic vinaigrette r...,field greens caesar mediterranean spinach arug...,pizza italian,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4b cafe,stuffed with cream cheese. our freshly...,Chips And Salsa Chips And Nacho Cheese Sauce C...,stuff cream cheese freshly fry corn to...,chip and salsa chip and nacho cheese sauce chi...,mexican american,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5 r cha thai go,Finely sliced veggies and glass noodles wr...,Thai Iced Tea Thai Iced Coffee Iced Tea Soft D...,finely sliced veggie glass noodle wrap lig...,thai iced tea thai iced coffee iced tea soft d...,thai,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
# Set all newly created columns to 0 by default
filtered_allmenus_df[categories] = filtered_allmenus_df[categories].fillna(0)
filtered_allmenus_df.head(1)

,name,menu_desc,menu_titles,clean_menu_desc,clean_menu_titles,categories,vegan,asian,ribs,fine,...,fresh,puerto,gyros,cantonese,european,cream,cocktails,pub,indian,steak
0,#1 pho,Finely ground pork grilled on wooden skewer wi...,Grilled Meat Balls Vietnames Crispy Crepe Cris...,finely ground pork grill wooden skewer lettuce...,grilled meat ball vietname crispy crepe crispy...,vietnamese,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [191]:
# Loop through categories column
for row in filtered_allmenus_df.itertuples():
            
    filtered_categories = [] # Used for appending categories
    
    # Create list
    cats = row.categories.split(' ')
    
    for category in cats:
        
        # Get all food and drink related categories
        if category in categories:
            filtered_categories.append(category)
    
    filtered_allmenus_df.loc[row.Index, filtered_categories] = 1 # Set to 1 for all food/drink related categories

In [192]:
filtered_allmenus_df.head()

,name,menu_desc,menu_titles,clean_menu_desc,clean_menu_titles,categories,vegan,asian,ribs,fine,...,fresh,puerto,gyros,cantonese,european,cream,cocktails,pub,indian,steak
0,#1 pho,Finely ground pork grilled on wooden skewer wi...,Grilled Meat Balls Vietnames Crispy Crepe Cris...,finely ground pork grill wooden skewer lettuce...,grilled meat ball vietname crispy crepe crispy...,vietnamese,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24th street pizza & gyros,"Tomatoes, onions and tzatziki sauce. Lettuce, ...",Gyros Pita Chicken Gyros Pita Calzone Fried Zu...,tomato onion tzatziki sauce lettuce tomato oni...,gyros pita chicken gyros pita calzone fried zu...,sandwiches wings pizza american italian,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3 tomatoes & a mozzarella,Small side salad of greens&comma dressed with ...,Field Greens Caesar Mediterranean Spinach & Ar...,small salad dress house balsamic vinaigrette r...,field greens caesar mediterranean spinach arug...,pizza italian,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4b cafe,stuffed with cream cheese. our freshly...,Chips And Salsa Chips And Nacho Cheese Sauce C...,stuff cream cheese freshly fry corn to...,chip and salsa chip and nacho cheese sauce chi...,mexican american,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5 r cha thai go,Finely sliced veggies and glass noodles wr...,Thai Iced Tea Thai Iced Coffee Iced Tea Soft D...,finely sliced veggie glass noodle wrap lig...,thai iced tea thai iced coffee iced tea soft d...,thai,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Save filtered menu dataset

In [193]:
filtered_allmenus_df.to_csv('./dataset/filtered_menu.csv')

In [ ]:
# Saving food categories for later use
with open('./dataset/food_categories.pkl', 'wb') as f:
    pickle.dump(categories, f)

### Summary of Allmenus dataset
 - Before extracting data through webscraping, I retrieved US states and its respective cities that appears on Yelp's dataset.
 - Used BeautifulSoup in extracting restaurant's menu data from allmenus using Yelp's states and cities information.
 - Filtered restaurants that appeared on both Yelp's business and Allmenus' menu dataset which resulted in collecting 1199 unique restaurants.
     - Restaurants that appeared on both allmenus and Yelp's business dataset resulted in the following:
        - Removed 150,439 restaurant data from Yelp's business dataset (97% reduction).
        - Removed 1,040,494 restaurants from allmenus data (76.8% reduction).
        - Resulted in having 1199 unique restaurants.
 - Consolidated Allmenus' menu dataframe's menu description and menu titles based on restaurant name and its categories resulting in 1199 rows with 4 columns.
 - Creates categories columns based on Allmenus' categories values (ex: There are Japanese, Indian columns, and etc.)

---

## 3) Clean Yelp's Business, Review, Tips Datasets
Yelp’s review and tip has rich information about restaurants’ attributes and users’ feelings towards the restaurants they dined in.  Unlike Allmenus’ dataset - it does not have direct food information but rather general text value about the restaurant. Therefore, I thought it’ll be useful in identifying the restaurant using text value provided in reviews and tips. Most of the columns will be removed from the business dataset as new columns will be created using Allmenus' categories columns as it has valuable information regarding restaurant's cuisine. Reviews will be filtered based on restaurants available in the filtered business dataset.

1. Remove all columns except business_id and restaurant name.
2. Filter review  and dataset by business id.
3. Text preprocessing - tokenizing, lemmatizing, lowercasing, and removing stop words.
4. Group all reviews/tips by restaurant name and remove duplicates.
5. Combine text (reviews/tips), business, and allmenus dataframes.
6. Identify cosine similarities between all restaurants.

### 3.1 Remove all columns except business_id and restaurant name

In [194]:
# Remove all columns except business_id and restaurant name
business_df = business_df[['business_id', 'name']].reset_index(drop=True)
business_df.head()

,business_id,name
0,Sd75ucXKoZUM2BEfBHFUOg,china gourmet
1,j9bWpCRwpDVfwVT_V85qeA,papaya thai
2,MTx-Zdl_KcU_z9G832XAjg,taco bell
3,UZ34XAw1GJ3chCOuKL69Mg,garcia's
4,UITPqkoDytnHT4kxaAyDeA,hibachi express


### 3.2 Filter review  and dataset by business id
- Remove columns that are not needed for NLP project.
- Remove rows with NaNs.
- Remove all reviews and tips that are not associated with restaurants in filtered business dataframe.

### Reviews and Tips

In [195]:
review_df.shape

(8021122, 9)

In [196]:
tip_df.shape

(1320761, 5)

In [197]:
review_df.head()

,funny,review_id,useful,date,business_id,text,cool,stars,user_id
0,0,xQY8N_XvtGbearJ5X4QryQ,5,2015-04-15 05:21:16,-MhfebM0QIsKt87iDN-FNw,"As someone who has worked with many museums, I...",0,2.0,OwjRMXRC0KyPrIlcjaXeFQ
1,1,UmFMZ8PyXZTY2QcwzsfQYA,1,2013-12-07 03:16:52,lbrU8StCq3yDfr-QMnGrmQ,I am actually horrified this place is still in...,0,1.0,nIJD_7ZXHq-FX8byPMOkMQ
2,0,LG2ZaYiOgpr2DK_90pYjNw,1,2015-12-05 03:18:11,HQl28KMwrEKHqhFrrDqVNQ,I love Deagan's. I do. I really do. The atmosp...,0,5.0,V34qejxNsCbcgD8C0HVk-Q
3,0,i6g_oA9Yf9Y31qt0wibXpw,0,2011-05-27 05:30:52,5JxlZaqCnk1MnbgRirs40Q,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",0,1.0,ofKDkJKXSKZXu5xJNGiiBQ
4,0,6TdNDKywdbjoTkizeMce8A,0,2017-01-14 21:56:57,IS4cv902ykd8wj1TR0N3-A,"Oh happy day, finally have a Canes near my cas...",0,4.0,UgMW8bLE0QMJDCkQ1Ax5Mg


In [198]:
tip_df.head()

,compliment_count,date,business_id,text,user_id
0,0,2013-11-26 18:20:08,UYX5zL_Xj9WEc_Wp-FrqHw,Here for a quick mtg,hf27xTME3EiCp6NL6VtWZQ
1,0,2014-06-15 22:26:45,Ch3HkwQYv1YKw_FO06vBWA,Cucumber strawberry refresher,uEvusDwoSymbJJ0auR3muQ
2,0,2016-07-18 22:03:42,rDoT-MgxGRiYqCmi0bG10g,Very nice good service good food,AY-laIws3S7YXNl_f_D6rQ
3,0,2014-06-06 01:10:34,OHXnDV01gLokiX1ELaQufA,It's a small place. The staff is friendly.,Ue_7yUlkEbX4AhnYdUfL7g
4,0,2011-04-08 18:12:01,GMrwDXRlAZU2zj5nH6l4vQ,"8 sandwiches, $24 total...what a bargain!!! An...",LltbT_fUMqZ-ZJP-vJ84IQ


#### 3.2.1 Remove columns that are not needed for NLP project

In [199]:
# Remove all except business_id and text
review_df = review_df[['business_id', 'text']]

In [200]:
tip_df = tip_df[['business_id', 'text']]

#### 3.2.2 Remove rows with NaNs

In [201]:
# Check for missing values
review_df.isna().sum()

business_id    0
text           2
dtype: int64

In [202]:
tip_df.isna().sum()

business_id    0
text           8
dtype: int64

In [203]:
review_df[review_df['text'].isna()]

,business_id,text
3529709,oXoVJ0xKv82cBo9U6oEjlQ,NaN
6502433,I7XWtKL1oHohZCtCCyQIog,NaN


In [204]:
# Dropping rows with NaN as I can't do much without text involved
review_df.dropna(inplace=True)
review_df.reset_index(inplace=True)

tip_df.dropna(inplace=True)
tip_df.reset_index(inplace=True)

#### 3.2.3 Remove all reviews and tips that are not associated with restaurants in filtered business dataframe.

In [205]:
# Get reviews that are in business dataframe
m = review_df['business_id'].isin(business_df['business_id'])
review_df = review_df[m].reset_index(drop=True)
review_df.shape

(367770, 3)

In [206]:
# Get tips that are in business dataframe
m = tip_df['business_id'].isin(business_df['business_id'])
tip_df = tip_df[m].reset_index(drop=True)
tip_df.shape

(70834, 3)

Dropped reviews from 8,021,122 to 367,770 rows (95.5% reduction) and dropped tips from 1,320,761 to 70,834 (95% reduction)

### 3.3 Text Preprocessing
- Tokenizing, Lemmatizing, lowerasing, and removing stop words.

In [207]:
review_df['clean_text'] = review_df['text'].apply(lambda text: tokenize(text))

In [208]:
tip_df['clean_text'] = tip_df['text'].apply(lambda text: tokenize(text))

### 3.4 Group all reviews/tips by restaurant name and remove duplicates.
- Concat both tips and reviews dataframe and groupby business_id and combine all texts.
- Add restaurant name to combined reviews and tips dataframe by using business dataframe's id.
    - Groupby restaurant name and combine text again.

#### 3.4.1 Concat both tips and reviews dataframe and groupby business_id and combine all texts

In [209]:
# Concat reviews and tips
text_df = pd.concat([review_df, tip_df])

In [210]:
text_df.shape

(438604, 4)

In [211]:
text_df = text_df[['business_id', 'text', 'clean_text']]
text_df.head(3)

,business_id,text,clean_text
0,d4qwVw4PcN-_2mK2o1Ro1g,10pm on a super bowl Sunday and they're alread...,p.m. super bowl sunday close weak wonder hard ...
1,d4qwVw4PcN-_2mK2o1Ro1g,Holy heck this place is amazing. I love their ...,holy heck -pron- chicken taco far favorite gre...
2,wkzWdo1mBqbzR2KPoXtWZw,Was a Chicago style deep dish. Homemade type ...,be chicago style deep dish homemade type crust...


In [212]:
# Consolidate all ingredients using groupby method
text_df = text_df.groupby('business_id', as_index=False).agg({'text': ' '.join, 'clean_text': ' '.join})
text_df.shape

(3404, 3)

#### 3.4.2 Add restaurant name to combined reviews and tips dataframe by using business dataframe's id

In [213]:
text_df['name'] = text_df['business_id'].apply(lambda x: business_df[business_df['business_id'] == x]['name'].values[0])

In [214]:
# Groupby name column and combine text
text_df = text_df.groupby('name', as_index=False).agg({'text': ' '.join, 'clean_text': ' '.join})
text_df.shape

(1199, 3)

In [215]:
text_df.head(3)

,name,text,clean_text
0,#1 pho,Fantastic pho! I had the vegetable pho in the ...,fantastic pho -pron- vegetable pho vegetable b...
1,24th street pizza & gyros,It's too bad this place has changed ownership....,-pron- bad change ownership -pron- stop day ad...
2,3 tomatoes & a mozzarella,"A cute, unassuming little bistro with excellen...",a cute unassume little bistro pizza go friday ...


All reviews and tips are combined together based on 1199 unique restaurants.

### 3.5 Combine text (reviews/tips) and allmenus dataframes
Merge two dataframes on 'name' column

In [216]:
restaurant_df = text_df.merge(filtered_allmenus_df, on='name')
restaurant_df.head(3)

,name,text,clean_text,menu_desc,menu_titles,clean_menu_desc,clean_menu_titles,categories,vegan,asian,...,fresh,puerto,gyros,cantonese,european,cream,cocktails,pub,indian,steak
0,#1 pho,Fantastic pho! I had the vegetable pho in the ...,fantastic pho -pron- vegetable pho vegetable b...,Finely ground pork grilled on wooden skewer wi...,Grilled Meat Balls Vietnames Crispy Crepe Cris...,finely ground pork grill wooden skewer lettuce...,grilled meat ball vietname crispy crepe crispy...,vietnamese,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24th street pizza & gyros,It's too bad this place has changed ownership....,-pron- bad change ownership -pron- stop day ad...,"Tomatoes, onions and tzatziki sauce. Lettuce, ...",Gyros Pita Chicken Gyros Pita Calzone Fried Zu...,tomato onion tzatziki sauce lettuce tomato oni...,gyros pita chicken gyros pita calzone fried zu...,sandwiches wings pizza american italian,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3 tomatoes & a mozzarella,"A cute, unassuming little bistro with excellen...",a cute unassume little bistro pizza go friday ...,Small side salad of greens&comma dressed with ...,Field Greens Caesar Mediterranean Spinach & Ar...,small salad dress house balsamic vinaigrette r...,field greens caesar mediterranean spinach arug...,pizza italian,0,0,...,0,0,0,0,0,0,0,0,0,0


In [217]:
restaurant_df.isna().sum()

name           0
text           0
clean_text     0
menu_desc      0
menu_titles    0
              ..
cream          0
cocktails      0
pub            0
indian         0
steak          0
Length: 119, dtype: int64

In [218]:
restaurant_df.shape

(1199, 119)

In [219]:
restaurant_df.head(3)

,name,text,clean_text,menu_desc,menu_titles,clean_menu_desc,clean_menu_titles,categories,vegan,asian,...,fresh,puerto,gyros,cantonese,european,cream,cocktails,pub,indian,steak
0,#1 pho,Fantastic pho! I had the vegetable pho in the ...,fantastic pho -pron- vegetable pho vegetable b...,Finely ground pork grilled on wooden skewer wi...,Grilled Meat Balls Vietnames Crispy Crepe Cris...,finely ground pork grill wooden skewer lettuce...,grilled meat ball vietname crispy crepe crispy...,vietnamese,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24th street pizza & gyros,It's too bad this place has changed ownership....,-pron- bad change ownership -pron- stop day ad...,"Tomatoes, onions and tzatziki sauce. Lettuce, ...",Gyros Pita Chicken Gyros Pita Calzone Fried Zu...,tomato onion tzatziki sauce lettuce tomato oni...,gyros pita chicken gyros pita calzone fried zu...,sandwiches wings pizza american italian,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3 tomatoes & a mozzarella,"A cute, unassuming little bistro with excellen...",a cute unassume little bistro pizza go friday ...,Small side salad of greens&comma dressed with ...,Field Greens Caesar Mediterranean Spinach & Ar...,small salad dress house balsamic vinaigrette r...,field greens caesar mediterranean spinach arug...,pizza italian,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Ensuring there are no whitespace in any of the text
Below is the example of extra white space found in the text.

In [220]:
restaurant_df['clean_menu_desc'][0]

'finely ground pork grill wooden skewer lettuce rice paper pan seared crepe pork bean sprout lightly batter shrimp sweet sour sauce ground onion mushroom fry crispy shell serve vegetarian roll request grill finely grind pork ball roll rice paper sweet sour sauce rice fresh lettuce bean sprout roll rice paper serve dipping sauce shredded rice fresh lettuce roll rice paper marinated charbroile fresh lettuce bacon sprout roll rice paper serve dipping sauce fresh lightly salt taste whole wing butter fry thinly sliced pork shrimp shred cabbage serve dip sauce peanut shred chicken cabbage toss peanut shrimp lotus root toss peanut thinly sliced flank steak marinate lime juice top peanut minced grill perfection sugar cane rice noodle rare beef       beef stew serve fresh french bread dipping egg noodle serve beef stew rice noodle serve beef stew clear noodle squid pork serve tho style clear noodle light broth serve chicken clear noodle squid fishball egg noodles wonton serve delicately season 

In [221]:
import re

# remove additional space from string  
restaurant_df['clean_text'] = restaurant_df['clean_text'].apply(lambda text: re.sub(' +', ' ', text))
restaurant_df['clean_menu_desc'] = restaurant_df['clean_menu_desc'].apply(lambda text: re.sub(' +', ' ', text))
restaurant_df['clean_menu_titles'] = restaurant_df['clean_menu_titles'].apply(lambda text: re.sub(' +', ' ', text))

#### Result

In [222]:
restaurant_df['clean_menu_desc'][0]

'finely ground pork grill wooden skewer lettuce rice paper pan seared crepe pork bean sprout lightly batter shrimp sweet sour sauce ground onion mushroom fry crispy shell serve vegetarian roll request grill finely grind pork ball roll rice paper sweet sour sauce rice fresh lettuce bean sprout roll rice paper serve dipping sauce shredded rice fresh lettuce roll rice paper marinated charbroile fresh lettuce bacon sprout roll rice paper serve dipping sauce fresh lightly salt taste whole wing butter fry thinly sliced pork shrimp shred cabbage serve dip sauce peanut shred chicken cabbage toss peanut shrimp lotus root toss peanut thinly sliced flank steak marinate lime juice top peanut minced grill perfection sugar cane rice noodle rare beef beef stew serve fresh french bread dipping egg noodle serve beef stew rice noodle serve beef stew clear noodle squid pork serve tho style clear noodle light broth serve chicken clear noodle squid fishball egg noodles wonton serve delicately season broth 

#### Combine clean menu description and titles together

In [223]:
restaurant_df['menu'] = restaurant_df['clean_menu_desc'] + restaurant_df['clean_menu_titles']

# remove additional space from string  
restaurant_df['menu'] = restaurant_df['menu'].apply(lambda text: re.sub(' +', ' ', text))

# Drop unneeded columns
restaurant_df = restaurant_df.drop(columns=['clean_menu_desc', 'clean_menu_titles', 'text', 'menu_desc', 
                                            'menu_titles', 'categories'])
# Re-order columns
first_cols = ['name', 'clean_text', 'menu']
second_cols = []
for col in restaurant_df.columns:
    if col not in first_cols:
        second_cols.append(col)
    
restaurant_df = restaurant_df[first_cols + second_cols]

# final dataset
restaurant_df.head()

,name,clean_text,menu,vegan,asian,ribs,fine,tapas,bistro,szechwan,...,fresh,puerto,gyros,cantonese,european,cream,cocktails,pub,indian,steak
0,#1 pho,fantastic pho -pron- vegetable pho vegetable b...,finely ground pork grill wooden skewer lettuce...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24th street pizza & gyros,-pron- bad change ownership -pron- stop day ad...,tomato onion tzatziki sauce lettuce tomato oni...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3 tomatoes & a mozzarella,a cute unassume little bistro pizza go friday ...,small salad dress house balsamic vinaigrette r...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4b cafe,good consensus average home fairly easily emul...,stuff cream cheese freshly fry corn tortilla ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5 r cha thai go,try r cha today turn permanently close very sa...,finely sliced veggie glass noodle wrap lightl...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [225]:
# Remove general term 'food' from the list
restaurant_df = restaurant_df.drop(columns=['food'])

In [233]:
# Rename (traditional) and (new) to traiditonal and new
restaurant_df = restaurant_df.rename(columns={"(traditional)": "traditional", "(new)": "new"})

In [234]:
restaurant_df.to_csv('./dataset/restaurants_data.csv')

### Summary of Yelp Dataset
Most of the columns in Yelp's business, review, and tips dataframes have been removed except for business_id and text column. Grouped both review and tips dataframe and began grouping based on business_id and restaurant name which resulted in consolidating all text into one row per restaurant. After removing duplicates on all dataframes, I merged consolidated yelp's dataframe with allmenus dataframe. Merged data frame has all the necessary information in identifying the restaurant as it has menu description, menu titles, restaurant cuisine attributes, and users inputted text.

**Snapshot**
1. Dropped reviews from 8,021,122 to 367,770 rows (95.5% reduction) and dropped tips from 1,320,761 to 70,834 (95% reduction)
2. Combined reviews and tips dataframe which accumulated up to 438,604; however after grouping based on business_id and restaurant name - it dropped to 1199.
3. Dropped rows with NaNs in the text column.
4. Merged cleaned Yelp's dataframe with Allmenus dataframe on restaurant name.
5. Tokenized all text and removed any whitespace occurring on text columns.